In [1]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel # Essential for parallel execution # will cover more on this in the next chapter

# Load environment variables from a .env file. 🌍
# This ensures your API keys for OpenAI and Anthropic are loaded securely.
load_dotenv()

# Initialize the OpenAI Chat model. 🤖
# This model will be used for generating notes and for the final merging step.
model1 = ChatOpenAI()

# Initialize the Anthropic Chat model. 🤖
# This model will be used for generating the quiz questions and answers.
# Using a specific model like 'claude-3-7-sonnet-20250219' for Anthropic.
model2 = ChatAnthropic(model_name='claude-3-7-sonnet-20250219')

# Define the first prompt template for generating notes. 📝
# It takes 'text' as input and asks for short, simple notes.
prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

# Define the second prompt template for generating quiz questions. 📝
# It also takes 'text' as input and asks for 5 short question-answers.
prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

# Define the third prompt template for merging the results. 📝
# This prompt takes two distinct inputs: 'notes' and 'quiz', and asks the LLM
# to combine them into a single coherent document.
prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

# Initialize a StrOutputParser. 📄
# This parser is used after each model call to extract the raw string content,
# ensuring that the output passed between components is clean text.
parser = StrOutputParser()

# Define the PARALLEL CHAIN using RunnableParallel. 👯
# This is the core of the parallel execution.
# - It takes a dictionary where keys are the output names and values are LangChain Runnables.
# - Both 'notes' (prompt1 | model1 | parser) and 'quiz' (prompt2 | model2 | parser)
#   will receive the *same initial input* (`text`) and execute concurrently.
# - The output of `parallel_chain` will be a dictionary: `{'notes': '...', 'quiz': '...'}`.
parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser, # Generate notes using model1
    'quiz': prompt2 | model2 | parser   # Generate quiz using model2 (concurrently)
})

# Define the MERGE CHAIN. ➡️
# This is a simple sequential chain that takes the combined output of the
# `parallel_chain` (which is a dictionary with 'notes' and 'quiz' keys)
# and feeds it into `prompt3` for merging.
# The `prompt3` expects inputs named 'notes' and 'quiz', which perfectly matches
# the output keys of `parallel_chain`.
merge_chain = prompt3 | model1 | parser

# Combine the parallel chain and the merge chain into a single, overall chain. 🔗
# The output of `parallel_chain` (the dictionary of notes and quiz) is piped
# directly as input to `merge_chain`.
chain = parallel_chain | merge_chain

# The input text to be processed.
text = """Linear Regression is a fundamental supervised learning algorithm used for predictive analysis. It models the relationship between a dependent variable and one or more independent variables by fitting a linear equation to the observed data. The goal is to find the best-fitting straight line (or hyperplane in higher dimensions) that minimizes the sum of squared differences between the observed and predicted values.

Advantages:
Simplicity and Interpretability: Easy to understand and implement, and the coefficients provide clear insights into the relationship between variables.
Computational Efficiency: Relatively fast to train, especially on large datasets.
Good Baseline: Often serves as a strong baseline model for comparison with more complex algorithms.

Disadvantages:
Assumes Linearity: It assumes a linear relationship between the dependent and independent variables, which may not always hold true in real-world data.
Sensitive to Outliers: Outliers can significantly impact the regression line and model performance.
Homoscedasticity Assumption: Assumes that the variance of the errors is constant across all levels of the independent variables.
Multicollinearity: High correlation between independent variables can make coefficient interpretation difficult and lead to unstable models.
"""

# Invoke the entire chain with the input text. 🚀
# The `text` is passed to `parallel_chain`, which then executes its branches concurrently.
# Once both branches complete, their outputs are collected and passed to `merge_chain`,
# which then produces the final combined document.
result = chain.invoke({'text':text})

# Print the final merged result. 📊
print(result)

# Print an ASCII representation of the chain's graph. 📈
# This visualizes the entire workflow, clearly showing the parallel branches
# for 'notes' and 'quiz' originating from the initial 'text' input,
# and then converging into the 'merge_chain'. This is invaluable for
# understanding complex chain structures.
chain.get_graph().print_ascii()

Linear Regression

- Linear Regression is a supervised learning algorithm for predictive analysis.
- It models the relationship between dependent and independent variables with a linear equation.
- Advantages of Linear Regression include simplicity, interpretability, computational efficiency, and serving as a good baseline model.
- Disadvantages of Linear Regression include assuming linearity, sensitivity to outliers, the homoscedasticity assumption, and multicollinearity.

Quiz Questions:

1. **Q: What is the main goal of Linear Regression?**
   A: The main goal of Linear Regression is to find the best-fitting straight line (or hyperplane in higher dimensions) that minimizes the sum of squared differences between observed and predicted values.

2. **Q: Name two advantages of Linear Regression.**
   A: Two advantages of Linear Regression are its simplicity and interpretability, and its computational efficiency, especially on large datasets.

3. **Q: How does Linear Regression handle ou